In [78]:
with open(file="/workspaces/Break_Projects/data/ssp256.txt", mode="r") as f:
    ssp256_split: list[str] = f.read().split(sep='\n')
    radius: float = float(ssp256_split[0])
    coordset: list[str] = ssp256_split[1:-1]
    coords: list[list[float]] = [[float(x), float(y), float(z)] for _, x, y, z in [c.split() for c in coordset]]
    print(coords) 

[[0.131466250332, 0.111117417957, -0.071326383627], [0.010847665986, 0.090666516829, 0.162800718411], [-0.132227287147, 0.118659788177, -0.057491171364], [0.131639421017, -0.119418785234, 0.057652727803], [-0.011435532116, -0.091425513886, -0.162639161971], [-0.132054116462, -0.111876415014, 0.071487940067], [-0.229257916562, -0.266229893774, -0.156384187803], [0.210050337001, -0.281778151675, -0.156421901554], [-0.011078122273, -0.313634536988, 0.222600318994], [-0.210076865493, 0.282027627919, 0.156315504898], [0.229491363078, 0.266888322756, 0.156113862924], [0.011199009109, 0.313907510331, -0.222786465409], [-0.013679416663, -0.39234878714, -0.041059551465], [0.014151520081, 0.392501578547, 0.041101423281], [-0.346233052898, 0.029133854206, 0.209510300649], [0.346257219767, -0.029754717547, -0.209665543022], [0.35330931195, 0.005598157939, 0.200221047869], [-0.353601095309, -0.005220371046, -0.200117665412], [-0.002752745172, 0.080877081607, -0.398233481808], [0.002745536316, -0.08

In [79]:
import numpy as np

def generate_sphere_faces(resolution: int = 20) -> np.ndarray:
    """
    Generate the faces of a sphere as triangles.

    Name
    ----
    generate_sphere_faces

    Description
    -----------
    This function generates a set of triangular faces that approximate a sphere. 
    The faces are computed based on a grid of specified resolution, allowing 
    for a smoother or coarser representation of the sphere.

    Parameters
    ----------
    resolution : int, optional
        The resolution of the sphere mesh, defining the number of subdivisions 
        along each axis. Higher values result in more triangles and a smoother 
        appearance, by default 20.

    Returns
    -------
    np.ndarray
        An array of shape (N, 3) where N is the number of triangular faces. 
        Each row represents the indices of the vertices that form a triangle.

    Examples
    --------
    >>> faces = generate_sphere_faces(resolution=10)
    >>> faces.shape
    (180, 3)
    """
    # Create indices for a grid of shape (resolution - 1, resolution - 1)
    i, j = np.indices(dimensions=(resolution - 1, resolution - 1))
    
    # Calculate the vertex indices for each corner of the quad
    top_left = i * resolution + j
    top_right = top_left + 1
    bottom_left = top_left + resolution
    bottom_right = bottom_left + 1
    
    # Create two triangles for each quad
    faces = np.vstack(tup=(
        np.column_stack(tup=(top_left.ravel(), top_right.ravel(), bottom_left.ravel())),
        np.column_stack(tup=(top_right.ravel(), bottom_right.ravel(), bottom_left.ravel()))
    ))
    
    return faces


In [80]:
import numpy as np

def generate_sphere_points(center: np.ndarray, radius: float = 0.1, resolution: int = 20) -> np.ndarray:
    """
    Generate points on the surface of a sphere.

    Name
    ----
    generate_sphere_points

    Description
    -----------
    This function generates a set of points uniformly distributed on the surface 
    of a sphere. The points are computed based on spherical coordinates and then 
    converted to Cartesian coordinates, offset by a specified center.

    Attributes
    ----------
    center : np.ndarray
        A 1D array representing the (x, y, z) coordinates of the sphere's center.

    Parameters
    ----------
    center : np.ndarray
        The center of the sphere as a 1D array of shape (3,), representing 
        the coordinates in 3D space.
    radius : float, optional
        The radius of the sphere, by default 0.1. A larger radius results in 
        points further from the center.
    resolution : int, optional
        The number of points generated along each axis of the sphere, by 
        default 20. Higher values create a denser distribution of points.

    Returns
    -------
    np.ndarray
        An array of shape (N, 3) where N is the total number of points generated 
        on the sphere. Each row represents the (x, y, z) coordinates of a point.

    Examples
    --------
    >>> center = np.array([1.0, 2.0, 3.0])
    >>> points = generate_sphere_points(center=center, radius=0.5, resolution=10)
    >>> points.shape
    (100, 3)
    """
    # Create a grid of theta (0 to pi) and phi (0 to 2pi)
    theta, phi = np.meshgrid(
        np.linspace(start=0, stop=np.pi, num=resolution), 
        np.linspace(start=0, stop=2 * np.pi, num=resolution)
    )
    
    # Calculate Cartesian coordinates for the points on the sphere
    x = radius * np.sin(theta) * np.cos(phi)
    y = radius * np.sin(theta) * np.sin(phi)
    z = radius * np.cos(theta)
    
    # Stack the coordinates and add the center offset
    sphere_points = np.column_stack(tup=(x.ravel(), y.ravel(), z.ravel())) + center
    
    return sphere_points


In [81]:
from scipy.stats import uniform

def generate_random_points_in_sphere(center: np.ndarray, radius: float, radial_scale: float, num_points: int) -> np.ndarray:
    """
    Generate random points uniformly distributed within a sphere.

    Name
    ----
    generate_random_points_in_sphere

    Description
    -----------
    This function generates a specified number of random points that are uniformly 
    distributed within a sphere of a given radius. The points are generated in spherical 
    coordinates and then converted to Cartesian coordinates, offset by a specified center.

    Attributes
    ----------
    center : np.ndarray
        A 1D array representing the (x, y, z) coordinates of the sphere's center.

    Parameters
    ----------
    center : np.ndarray
        The center of the sphere as a 1D array of shape (3,), representing 
        the coordinates in 3D space.
    radius : float
        The radius of the sphere within which the points will be generated. 
        The points will be uniformly distributed within this volume.
    radial_scale : float
        A scaling factor for the radial distance that adjusts the distribution of 
        points within the sphere. A value of 1 maintains uniformity, while values 
        less than 1 compress the points toward the center.
    num_points : int
        The total number of random points to generate within the sphere.

    Returns
    -------
    np.ndarray
        An array of shape (N, 3) where N is the number of points generated. 
        Each row represents the (x, y, z) coordinates of a point.

    Examples
    --------
    >>> center = np.array([0.0, 0.0, 0.0])
    >>> points = generate_random_points_in_sphere(center=center, radius=1.0, radial_scale=1.0, num_points=100)
    >>> points.shape
    (100, 3)
    """
    # Generate random spherical coordinates
    u = uniform.rvs(size=num_points)  # Uniformly distributed random values for radius
    cos_theta = uniform.rvs(loc=-1, scale=2, size=num_points)  # Cosine of the polar angle
    phi = uniform.rvs(scale=2 * np.pi, size=num_points)  # Azimuthal angle

    # Compute spherical coordinates
    r = radius * np.cbrt(u) * radial_scale  # Radial distance
    theta = np.arccos(cos_theta)  # Polar angle

    # Convert spherical coordinates to Cartesian coordinates
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    # Stack and shift points by center
    points = np.vstack((x, y, z)).T + center

    return points


In [82]:
from typing import List
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import random

def generate_unique_colors(n: int) -> List[str]:
    """
    Generate a list of unique colors.

    Name
    ----
    generate_unique_colors

    Description
    -----------
    This function generates a specified number of unique colors by sampling 
    from the 'rainbow' colormap. The colors are returned in hexadecimal format 
    and are shuffled to ensure randomness.

    Attributes
    ----------
    n : int
        The number of unique colors to generate.

    Parameters
    ----------
    n : int
        The desired number of unique colors. Must be greater than zero.

    Returns
    -------
    List[str]
        A list of unique colors represented as hexadecimal strings.

    Examples
    --------
    >>> colors = generate_unique_colors(5)
    >>> len(colors)
    5
    >>> all(len(color) == 7 for color in colors)  # Check if all colors are in hex format
    True
    """
    # Generate a colormap from 'rainbow'
    cmap = plt.get_cmap(name='rainbow')

    # Generate equally spaced points in the colormap
    colors = [cmap(i / (n - 1)) for i in range(n)]

    # Convert colors to hexadecimal format
    unique_colors = [mcolors.rgb2hex(c=color) for color in colors]

    random.shuffle(x=unique_colors)
    return unique_colors


In [83]:
from scipy.stats import dirichlet
import numpy as np

def generate_dirichlet_unbalanced_amounts(number_of_values: int, total_sum: int, min_value: int) -> np.ndarray:
    """
    Generate unbalanced amounts using a Dirichlet distribution.

    Name
    ----
    generate_dirichlet_unbalanced_amounts

    Description
    -----------
    This function generates a set of unbalanced integer amounts that sum to a 
    specified total. It uses a Dirichlet distribution to create proportions, 
    scales them to the desired total sum, and ensures that each amount meets a 
    minimum value requirement.

    Parameters
    ----------
    number_of_values : int
        The number of values to generate.
    total_sum : int
        The desired total sum of the generated values.
    min_value : int
        The minimum value that each generated amount must meet.

    Returns
    -------
    np.ndarray
        An array of integers representing the generated amounts, which sum to 
        the specified total and respect the minimum value constraint.

    Examples
    --------
    >>> amounts = generate_dirichlet_unbalanced_amounts(number_of_values=5, total_sum=100, min_value=10)
    >>> amounts
    array([24, 22, 19, 15, 20])
    >>> np.sum(amounts)
    100
    >>> np.all(amounts >= 10)
    True
    """
    # Step 1: Generate Dirichlet distributed values
    dirichlet_values = dirichlet.rvs(alpha=np.ones(shape=number_of_values), size=1)[0]

    # Step 2: Scale these values to the target total sum minus the minimum value contribution
    scaled_values = dirichlet_values * number_of_values * (total_sum - min_value)

    # Step 3: Convert to integers while ensuring a minimum value
    floored_values = np.floor(scaled_values).astype(int)
    integer_values = floored_values + min_value

    # Step 4: Handle rounding errors to ensure the sum matches the total_sum
    # Calculate the difference between the desired sum and the current integer sum
    current_sum = np.sum(a=integer_values)
    difference = number_of_values * total_sum - current_sum

    # Get the decimal parts and their indices
    decimal_parts = scaled_values - floored_values
    # Sort indices based on the decimal parts in descending order
    sorted_indices = np.argsort(a=-decimal_parts)

    # Adjust the values with the largest decimal parts first
    for i in range(difference):
        integer_values[sorted_indices[i]] += 1

    return integer_values


In [84]:
import plotly.graph_objects as go

def plot_spheres(coords, radius: float, resolution: int = 30, avg_points_per_sphere: int = 1000, min_points_per_sphere: int = 100):

    sphere_faces = generate_sphere_faces(resolution=resolution)

    plot_data = []

    dirichlet_unbalanced_amounts = generate_dirichlet_unbalanced_amounts(number_of_values=256, total_sum=avg_points_per_sphere, min_value=min_points_per_sphere)

    X = np.empty(shape=(0, 3), dtype=float)

    unique_colors = generate_unique_colors(n=256)

    unit_sphere = generate_sphere_points(center=np.array(object=[0, 0, 0]), radius=1, resolution=resolution)

    coords_array = np.array(object=coords)

    flag = True
    plot_data.append(go.Mesh3d(
            x=unit_sphere[:, 0],
            y=unit_sphere[:, 1],
            z=unit_sphere[:, 2],
            i=sphere_faces[:, 0],
            j=sphere_faces[:, 1],
            k=sphere_faces[:, 2],
            opacity=0.5,
            color='blue',
            name='Unit Sphere',
            legendgroup='Unit Sphere',
            showlegend=flag
        ))

    for i, coord in enumerate(iterable=coords):
        sphere_points = generate_sphere_points(center=np.array(object=coord), radius=radius, resolution=resolution)
        plot_data.append(go.Mesh3d(
            x=sphere_points[:, 0],
            y=sphere_points[:, 1],
            z=sphere_points[:, 2],
            i=sphere_faces[:, 0],
            j=sphere_faces[:, 1],
            k=sphere_faces[:, 2],
            opacity=0.5,
            color=unique_colors[i],
            name='Inner Spheres',
            legendgroup='Inner Spheres',
            showlegend=flag
        ))

        # Generate random points inside the current sphere
        random_points = generate_random_points_in_sphere(center=np.array(object=coord), radius=radius, radial_scale=1, num_points=dirichlet_unbalanced_amounts[i])
        plot_data.append(go.Scatter3d(
            x=random_points[:, 0],
            y=random_points[:, 1],
            z=random_points[:, 2],
            mode='markers',
            marker=dict(size=2, color=unique_colors[i]),
            name='Random Points',
            legendgroup='Random Points',
            showlegend=flag
        ))

        X = np.concatenate((X, random_points), axis=0)

        flag = False
    
    y = np.array(object=[color for color, count in zip(unique_colors, dirichlet_unbalanced_amounts) for _ in range(count)])
    
    fig = go.Figure(data=plot_data)
    fig.update_layout(scene_aspectmode='data')
    fig.show()

    return X, y

In [86]:
X, y = plot_spheres(coords=coords, radius=radius)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed